In [ ]:
!pip install torchcodec
!pip install soundfile
!pip install scipy

In [ ]:
from diffwave.inference import predict as diffwave_predict
import torch
from scipy.io.wavfile import write
import numpy as np

# Perform the following to get the mel spectrograms

Check the preprocess file to see what mel spectrograms they use

In [2]:
!python -m diffwave.preprocess ./audio_wav

Preprocessing: 100%|██████████████████████████████| 1/1 [00:02<00:00,  2.43s/it]


In [9]:
mel = np.load("./audio_wav/LJ001-0003.wav.spec.npy")  # shape: (n_mels, time)

# # Convert to PyTorch tensor
mel = torch.from_numpy(mel).float()


model_dir = './pretrained_model/diffwave-ljspeech-22kHz-1000578.pt'
audio, sample_rate = diffwave_predict(mel, model_dir, fast_sampling=False)

write("output.wav", sample_rate, audio.squeeze().cpu().numpy())

> /home/wongbr55/diffwave_expose_latent_space/src/diffwave/inference.py(63)predict()
     61     T = []
     62     for s in range(len(inference_noise_schedule)):
---> 63       for t in range(len(training_noise_schedule) - 1):
     64         if talpha_cum[t+1] <= alpha_cum[s] <= talpha_cum[t]:
     65           twiddle = (talpha_cum[t]**0.5 - alpha_cum[s]**0.5) / (talpha_cum[t]**0.5 - talpha_cum[t+1]**0.5)

50
50
50
50


In [8]:
print(audio.shape)

torch.Size([1, 213248])


## Steps to train to fit brain signals into diffusion pipeline
1. Get mel spectrograms of all of the GT sentences
2. Use those to get latent variables to train
3. Train model(s) to give use mel spectrograms and latent variables

### Notes on setup
- Must choose whether we use fast sampling or not